In [1]:
import torch
from torch.utils.data import DataLoader
from miditok import REMI
from miditok.pytorch_data import DatasetMIDI, DataCollator
from pathlib import Path
import json
import pandas as pd
import numpy as np
import random
import utils
import importlib

In [2]:
importlib.reload(utils)

<module 'utils' from '/mnt/nfs_share_magnet1/lafuente/symbolic_music/author-profiling/experiments/e17/utils.py'>

In [3]:
k=3
train_data=pd.read_csv(f'../../train data/k folds/type0_train_set_k{k}.csv')
validation_data=pd.read_csv(f'../../train data/k folds/type0_test_set_k{k}.csv')

## 1. Get features vector

### 1.1 Create dataloader

In [4]:
val_dataloader=utils.create_data_loader(scores_df=validation_data,paths_column_name='type0_path')
train_dataloader=utils.create_data_loader(scores_df=train_data,paths_column_name='type0_path')

/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: Argument nb_tempos has been renamed num_tempos, you should consider to updateyour code with this new argument name.
  return cls(**input_dict, **kwargs)


### 1.2 Get feature vectors from pre-trained model

In [5]:
train_feature_vectors=utils.get_feature_vectors(dataloader=train_dataloader,
                                                dataframe=train_data,
                                                set_type='train', 
                                                feature_tensors=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
computing feature tensors: 100%|██████████████| 184/184 [24:39<00:00,  8.04s/it]


In [6]:
train_feature_vectors

,0,1,2,3,4,5,6,7,8,9,...,1046007,1046008,1046009,1046010,1046011,1046012,1046013,1046014,1046015,label
0,0.420797,-0.562061,0.451700,-0.509399,-1.038532,0.375727,1.223970,-0.798628,1.117829,-1.437254,...,1.117981,5.796663,-0.649414,-3.239012,1.249011,-3.173556,2.111859,3.843078,5.030483,1
1,0.596922,-0.570097,0.610556,-0.578611,-1.018962,0.393136,1.160797,-1.027660,0.902073,-1.307048,...,2.739024,2.639503,0.782972,-3.066326,0.921386,-4.675134,0.556902,1.563904,5.277238,1
2,0.679969,-0.600057,0.566742,-0.927327,-0.950821,0.116811,0.851089,-1.076599,1.336228,-1.105483,...,1.842687,0.867163,3.827507,-4.332413,2.523508,-0.084526,-1.690104,-2.383358,4.141416,0
3,0.705700,-0.491180,0.735000,-0.482003,-1.033045,0.199081,0.842007,-0.933514,0.850870,-1.309024,...,-1.059271,2.744436,3.368528,-2.255012,6.145209,-5.029330,0.634608,0.256072,7.453728,0
4,0.468751,-0.533240,0.585627,-0.284875,-1.179047,0.351912,1.177288,-0.718073,0.914706,-1.245607,...,8.966378,5.424798,-3.183829,-7.840390,1.087433,-2.934630,1.692597,3.545684,4.300663,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,0.536290,-0.505152,0.640095,-0.310818,-1.300964,0.364821,1.202627,-0.753760,0.601215,-1.132895,...,-0.173257,2.075835,0.885273,-1.469112,1.081287,0.581385,3.273939,1.322427,2.514951,1
912,0.622229,-0.377938,0.752228,-0.352623,-1.229601,0.316694,1.192149,-0.778355,0.698013,-1.222824,...,3.455268,1.069326,0.243430,-3.820273,1.128624,-0.168324,1.081202,0.749186,1.328845,1
913,0.501605,-0.366239,0.595239,-0.315524,-1.000022,0.317505,1.234752,-0.921635,1.028411,-1.241344,...,3.454569,3.967445,-1.315043,-2.330245,1.431489,-1.097921,0.399926,4.033278,7.254939,1
914,0.795898,-0.528706,0.662959,-0.397001,-1.118110,0.148441,0.970396,-1.033166,0.977161,-1.330773,...,1.674139,-3.791064,1.436035,-6.432080,-1.150060,-3.084278,0.794212,2.412942,2.440402,1


In [7]:
val_feature_vectors=utils.get_feature_vectors(dataloader=val_dataloader,
                                                dataframe=validation_data,
                                                set_type='val', 
                                                feature_tensors=False)

computing feature tensors: 100%|████████████████| 46/46 [05:52<00:00,  7.67s/it]


In [8]:
val_feature_vectors

,0,1,2,3,4,5,6,7,8,9,...,1046007,1046008,1046009,1046010,1046011,1046012,1046013,1046014,1046015,label
0,0.517793,-0.389997,0.678372,-0.477860,-1.097905,0.324338,1.105011,-0.977612,1.075689,-1.390321,...,0.602549,5.651792,-1.148675,-1.002282,1.330335,-2.166102,-1.913405,3.282932,2.388539,1
1,0.596922,-0.570097,0.610556,-0.578611,-1.018962,0.393136,1.160797,-1.027660,0.902073,-1.307048,...,1.877814,-0.708320,-0.598502,0.250330,3.565836,-3.319880,1.684049,-1.760255,0.373773,1
2,0.596922,-0.570097,0.610556,-0.578611,-1.018962,0.393136,1.160797,-1.027660,0.902073,-1.307048,...,-0.806293,2.106970,-1.475589,1.870050,-1.242909,-4.205734,-3.044392,-3.653877,-2.477349,1
3,0.596922,-0.570097,0.610556,-0.578611,-1.018962,0.393136,1.160797,-1.027660,0.902073,-1.307048,...,-0.387686,3.351962,-4.443542,-0.686150,-0.802624,-9.529907,2.400610,-3.040552,3.516443,0
4,0.523632,-0.617408,0.608217,-0.260638,-1.003129,0.432548,1.117777,-0.715589,0.959216,-1.463569,...,2.695567,1.411335,-2.007365,0.046799,-2.632613,-1.531812,1.015892,0.744893,1.477613,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,0.596922,-0.570097,0.610556,-0.578611,-1.018962,0.393136,1.160797,-1.027660,0.902073,-1.307048,...,4.046533,3.323665,-1.989096,-5.728514,-2.333056,-3.528256,0.742845,1.602498,5.750250,0
225,0.596922,-0.570097,0.610556,-0.578611,-1.018962,0.393136,1.160797,-1.027660,0.902073,-1.307048,...,-2.717252,0.918007,1.667352,2.780399,-2.120891,-6.134084,0.851840,2.837955,-0.383224,0
226,0.596922,-0.570097,0.610556,-0.578611,-1.018962,0.393136,1.160797,-1.027660,0.902073,-1.307048,...,-2.362377,2.252582,2.867711,-0.995827,-0.014305,-2.727712,2.483629,-5.889081,2.127364,0
227,0.514466,-0.628554,0.614967,-0.323817,-1.030299,0.358691,1.160397,-0.782786,0.927562,-1.308334,...,2.764035,3.227076,-1.600685,-3.392083,0.955045,1.860130,2.925563,2.083425,4.142221,0


## 2. Train MLP

In [9]:
predictions_df_train_e17,metrics_df,predictions_df_val_e17=utils.train_MLP_classfier(train_feature_vectors=train_feature_vectors,
                          val_feature_vectors=val_feature_vectors,
                          k=k)

MLP(
  (fc1): Linear(in_features=1046016, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
)
Epoch 0 | Train Loss  10.77| Train Balanced Accuracy   0.54
Epoch 0 | Validation Loss   3.62| Validation Balanced Accuracy:   0.60
Epoch 1 | Train Loss   2.27| Train Balanced Accuracy   0.72
Epoch 1 | Validation Loss   5.01| Validation Balanced Accuracy:   0.53
Epoch 2 | Train Loss   0.56| Train Balanced Accuracy   0.89
Epoch 2 | Validation Loss   3.79| Validation Balanced Accuracy:   0.54
Epoch 3 | Train Loss   0.18| Train Balanced Accuracy   0.96
Epoch 3 | Validation Loss   2.33| Validation Balanced Accuracy:   0.59
Epoch 4 | Train Loss   0.05| Train Balanced Accuracy   0.98
Epoch 4 | Validation Loss   2.36| Validation Balanced Accuracy:   0.59
Epoch 5 | Train Loss   0.03| Train Balanced Accuracy   0.99
Epoch 5 | Valid

In [10]:
predictions_df_train_e17.to_csv(f'predictions_df_train_e17_k{k}_full_seqs.csv')
metrics_df.to_csv(f'metrics_df_e17_k{k}_full_seqs.csv',index=False)
predictions_df_val_e17.to_csv(f'predictions_df_test_e17_k{k}_full_seqs.csv')